In [1]:
import pandas as pd
import requests as res 
from bs4 import BeautifulSoup as bs 
import plotly
import re
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)


data=[] 
st1 = input("Enter first company code : ")
st2 = input("Enter senond company code : ")

#request for page of particular company's historical stock data site
req=res.get("https://in.finance.yahoo.com/quote/"+st1+"/history/",headers={"User-Agent":"chrome/127.0.0.1"})

#parse the requested unstructure data and convert into HTML tree format 
h = bs(req.content,"lxml")

#find the table in the parse HTML tree formated data and store into table_body variable
table_body=h.find("tbody") 

#find the each ROWS in the table_body and store in the rows variable
rows = table_body.find_all('tr')

i=0 
i1=0

#iterate the each row data using for loop
for tr in rows:
    df={}
    i=0 
    i1+=1 
    
    #find the each column data in the 'tr' variable and store in the 'cols' variable
    cols = tr.find_all('td')
    #if all condition satisfied 
    if(True):
        #iterate each 'cols' data and store into perticular list 
        for td in cols:
            
            #i th index = 0 contain the "DATE" , so extract i th element and store in the 'date' Dictionary
            if(i==0):
                df["date"]=td.text
            
            #i th index = 1 contain the "OPEN" , so extract i th element and store in the 'open' Dictionary
            if(i==1):
                df["open"]=(str(re.findall("\d+\.\d+",td.text)).strip("['']"))
            
            #i th index = 2 contain the "HIGH" , so extract i th element and store in the 'high' Dictionary
            if(i==2):
                df["high"]=(str(re.findall("\d+\.\d+",td.text)).strip("['']"))
            
            #i th index = 3 contain the "LOW" , so extract i th element and store in the 'low' Dictionary
            if(i==3):
                df["low"]=(str(re.findall("\d+\.\d+",td.text)).strip("['']"))
            
            #i th index = 4 contain the "CLOSE" , so extract i th element and store in the 'close' Dictionary
            if(i==4):
                df["close"]=(str(re.findall("[0-9]+.[0-9]+",td.text)).strip("['']"))
            
            #i th index = 6 contain the "VOLUME" , so extract i th element and store in the 'date' Dictionary
            if(i==6):
                s=""
                l=re.findall('[0-9]+',td.text)#REGEXP for find only Decimal number
                for x in l:
                    s=s+x
                df["volume"]=str(s)
            i+=1
            #store the dictionary into list (list of dictionry forms here)
            data.append(df)

#convert list of dictionry into DataFrame for make the data into table format
df1=pd.DataFrame(data)
df1 
#DataFrame into csv_file convertion
df1.to_csv("data_"+st1+".csv",index=False)

#make data list empty
data=[]

#request for page of particular company's historical stock data site
req=res.get("https://in.finance.yahoo.com/quote/"+st2+"/history/",headers={"User-Agent":"chrome/127.0.0.1"})

#parse the requested unstructure data and convert into HTML tree format 
h=bs(req.content,"lxml")

#find the table in the parse HTML tree formated data and store into table_body variable
table_body=h.find("tbody")

#find the each ROWS in the table_body and store in the rows variable
rows = table_body.find_all('tr')

i=0
i1=0

#iterate the each row data using for loop
for tr in rows:
    df={}
    i=0 
    i1+=1 
    
    #find the each column data in the 'tr' variable and store in the 'cols' variable
    cols = tr.find_all('td')
    
    #if all condition satisfied 
    if(True):
        #iterate each 'cols' data and store into perticular list 
        for td in cols:
            #i th index = 0 contain the "DATE" , so extract i th element and store in the 'date' Dictionary
            if(i==0):
                df["date"]=td.text
                
            #i th index = 1 contain the "OPEN" , so extract i th element and store in the 'open' Dictionary
            if(i==1):
                df["open"]=(str(re.findall("[0-9]+.[0-9]+",td.text)).strip("['']"))
            
            #i th index = 2 contain the "HIGH" , so extract i th element and store in the 'high' Dictionary
            if(i==2):
                df["high"]=(str(re.findall("[0-9]+.[0-9]+",td.text)).strip("['']"))
            
            #i th index = 3 contain the "LOW" , so extract i th element and store in the 'low' Dictionary
            if(i==3):
                df["low"]=(str(re.findall("[0-9]+.[0-9]+",td.text)).strip("['']"))
            
            #i th index = 4 contain the "CLOSE" , so extract i th element and store in the 'close' Dictionary
            if(i==4):
                df["close"]=(str(re.findall("[0-9]+.[0-9]+",td.text)).strip("['']"))
            
            #i th index = 6 contain the "VOLUME" , so extract i th element and store in the 'date' Dictionary
            if(i==6):
                s=""
                l=re.findall('[0-9]+',td.text)
                for x in l:
                    s=s+x
                df["volume"]=str(s)
            i+=1
            
            #store the dictionary into list (list of dictionry forms here)
            data.append(df)

#convert list of dictionry into DataFrame for make the data into table format
df2=pd.DataFrame(data) 
df2 
#DataFrame into csv_file convertion
df2.to_csv("data_"+st2+".csv",index=False)

#read first extracted and then converted csv file
csv_file1=pd.read_csv("data_"+st1+".csv",parse_dates=["date"],index_col="date").interpolate(method="time")

#read second extracted and then converted csv file
csv_file2=pd.read_csv("data_"+st2+".csv",parse_dates=["date"],index_col="date").interpolate(method="time")


#################################################################################################################

#################################### VISUALIZE DATA #############################################################

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['open'],name=st1+" open"),
          go.Scatter(x = csv_file1.index, y = csv_file1['close'],name=st1+" close")],
    layout=go.Layout(title="1.OPEN AND CLOSE COMPARISON OF "+st1)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['high'],name=st1+" high"),
          go.Scatter(x = csv_file1.index, y = csv_file1['low'],name=st1+" low")],
    layout=go.Layout(title="2.HIGH AND LOW COMPARISON OF "+st1)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file2.index, y = csv_file2['open'],name=st2+" open"),
          go.Scatter(x = csv_file2.index, y = csv_file2['close'],name=st2+" close")],
    layout=go.Layout(title="3.OPEN AND CLOSE COMPARISON OF "+st2)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file2.index, y = csv_file2['high'],name=st2+" high"),
          go.Scatter(x = csv_file2.index, y = csv_file2['low'],name=st2+" low")],
    layout=go.Layout(title="4.HIGH AND LOW COMPARISON OF "+st2)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['volume'],name=st1+" volume"),
          go.Scatter(x = csv_file2.index, y = csv_file2['volume'],name=st2+" volume")],
    layout=go.Layout(title="5.COMPARISON OF VOLUME STOCK TRADE BETWEEN "+st1+" and "+st2)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['close'],name=st1+" close"),
          go.Scatter(x = csv_file2.index, y = csv_file2['close'],name=st2+" close")],
    layout=go.Layout(title="6.COMPARISON OF CLOSE STOCK TRADE BETWEEN "+st1+" and "+st2)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['open'],name=st1+" open"),
          go.Scatter(x = csv_file2.index, y = csv_file2['open'],name=st2+" open")],
    layout=go.Layout(title=".7COMPARISON OF OPEN STOCK TRADE BETWEEN "+st1+" and "+st2)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['high'],name=st1+" high"),
          go.Scatter(x = csv_file2.index, y = csv_file2['high'],name=st2+" high")],
    layout=go.Layout(title="8.COMPARISON OF HIGH STOCK TRADE BETWEEN "+st1+" and "+st2)))

#for plot the graph we have to select table name and column name
plotly.offline.iplot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['low'],name=st1+" low"),
          go.Scatter(x = csv_file2.index, y = csv_file2['low'],name=st2+" low")],
    layout=go.Layout(title="9.COMPARISON OF LOW STOCK TRADE BETWEEN "+st1+" and "+st2)),
                    filename="COMPARISON OF LOW STOCK TRADE BETWEEN "+st1+" and "+st2)



#########################################################################################################

################################### SAVE GRATH INTO FILE SYSTEM #########################################


#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['open'],name=st1+" open"),
          go.Scatter(x = csv_file1.index, y = csv_file1['close'],name=st1+" close")],
    layout=go.Layout(title="1.OPEN AND CLOSE COMPARISON OF "+st1))
                   ,filename="OPEN AND CLOSE COMPARISON OF "+st1)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['high'],name=st1+" high"),
          go.Scatter(x = csv_file1.index, y = csv_file1['low'],name=st1+" low")],
    layout=go.Layout(title="2.HIGH AND LOW COMPARISON OF "+st1))
                   ,filename="HIGH AND LOW COMPARISON OF "+st1)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file2.index, y = csv_file2['open'],name=st2+" open"),
          go.Scatter(x = csv_file2.index, y = csv_file2['close'],name=st2+" close")],
    layout=go.Layout(title="3.OPEN AND CLOSE COMPARISON OF "+st2))
                   ,filename="OPEN AND CLOSE COMPARISON OF "+st2)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file2.index, y = csv_file2['high'],name=st2+" high"),
          go.Scatter(x = csv_file2.index, y = csv_file2['low'],name=st2+" low")],
    layout=go.Layout(title="4.HIGH AND LOW COMPARISON OF "+st2))
                   ,filename="HIGH AND LOW COMPARISON OF "+st2)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['volume'],name=st1+" volume"),
          go.Scatter(x = csv_file2.index, y = csv_file2['volume'],name=st2+" volume")],
    layout=go.Layout(title="5.COMPARISON OF VOLUME STOCK TRADE BETWEEN "+st1+" and "+st2))
                   ,filename="COMPARISON OF VOLUME STOCK TRADE BETWEEN "+st1+" and "+st2)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['close'],name=st1+" close"),
          go.Scatter(x = csv_file2.index, y = csv_file2['close'],name=st2+" close")],
    layout=go.Layout(title="6.COMPARISON OF CLOSE STOCK TRADE BETWEEN "+st1+" and "+st2))
                   ,filename="COMPARISON OF CLOSE STOCK TRADE BETWEEN "+st1+" and "+st2)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['open'],name=st1+" open"),
          go.Scatter(x = csv_file2.index, y = csv_file2['open'],name=st2+" open")],
    layout=go.Layout(title="7.COMPARISON OF OPEN STOCK TRADE BETWEEN "+st1+" and "+st2))
                   ,filename="COMPARISON OF OPEN STOCK TRADE BETWEEN "+st1+" and "+st2)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['high'],name=st1+" high"),
          go.Scatter(x = csv_file2.index, y = csv_file2['high'],name=st2+" high")],
    layout=go.Layout(title="8.COMPARISON OF HIGH STOCK TRADE BETWEEN "+st1+" and "+st2))
                   ,filename="COMPARISON OF HIGH STOCK TRADE BETWEEN "+st1+" and "+st2)

#for plot the graph we have to select table name and column name and save the graph in file
plotly.offline.plot(dict(
    data=[go.Scatter(x = csv_file1.index, y = csv_file1['low'],name=st1+" low"),
          go.Scatter(x = csv_file2.index, y = csv_file2['low'],name=st2+" low")],
    layout=go.Layout(title="9.COMPARISON OF LOW STOCK TRADE BETWEEN "+st1+" and "+st2)),
                    filename="COMPARISON OF LOW STOCK TRADE BETWEEN "+st1+" and "+st2)


#########################################################################################################



#for display both the tables we have to first concate 
dfs=pd.concat([df1,df2],keys=[st1,st2],axis=1)
# then display
dfs

Enter first company code : infy
Enter senond company code : tcs


D:\prog\Anaconda3\lib\site-packages\plotly\offline\offline.py:466: UserWarning:

Your filename `OPEN AND CLOSE COMPARISON OF infy` didn't end with .html. Adding .html to the end of your file.

D:\prog\Anaconda3\lib\site-packages\plotly\offline\offline.py:466: UserWarning:

Your filename `HIGH AND LOW COMPARISON OF infy` didn't end with .html. Adding .html to the end of your file.

D:\prog\Anaconda3\lib\site-packages\plotly\offline\offline.py:466: UserWarning:

Your filename `OPEN AND CLOSE COMPARISON OF tcs` didn't end with .html. Adding .html to the end of your file.

D:\prog\Anaconda3\lib\site-packages\plotly\offline\offline.py:466: UserWarning:

Your filename `HIGH AND LOW COMPARISON OF tcs` didn't end with .html. Adding .html to the end of your file.

D:\prog\Anaconda3\lib\site-packages\plotly\offline\offline.py:466: UserWarning:

Your filename `COMPARISON OF VOLUME STOCK TRADE BETWEEN infy and tcs` didn't end with .html. Adding .html to the end of your file.

D:\prog\Anaconda3\lib

infy                                                tcs               \
     close         date   high    low   open    volume  close         date   
0    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
1    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
2    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
3    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
4    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
5    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
6    10.55  25-Jan-2019  10.58  10.41  10.51   8839100   6.95  25-Jan-2019   
7    10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
8    10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
9    10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
10   10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
11   10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
12   10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
13   10.43  24-Jan-2019  10.55  10.43  10.53   6627700   6.63  24-Jan-2019   
14     NaN  24-Jan-2019    NaN    NaN  0.057       NaN   6.45  23-Jan-2019   
15     NaN  24-Jan-2019    NaN    NaN  0.057       NaN   6.45  23-Jan-2019   
16   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.45  23-Jan-2019   
17   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.45  23-Jan-2019   
18   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.45  23-Jan-2019   
19   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.45  23-Jan-2019   
20   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.45  23-Jan-2019   
21   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.20  22-Jan-2019   
22   10.53  23-Jan-2019  10.56  10.43  10.56   8108100   6.20  22-Jan-2019   
23   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.20  22-Jan-2019   
24   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.20  22-Jan-2019   
25   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.20  22-Jan-2019   
26   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.20  22-Jan-2019   
27   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.20  22-Jan-2019   
28   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.05  18-Jan-2019   
29   10.53  22-Jan-2019  10.68  10.47  10.64  10437700   6.05  18-Jan-2019   
..     ...          ...    ...    ...    ...       ...    ...          ...   
670  10.35  10-Sep-2018  10.48  10.30  10.38   4470300  10.83  07-Sep-2018   
671  10.35  10-Sep-2018  10.48  10.30  10.38   4470300  10.83  07-Sep-2018   
672  10.35  10-Sep-2018  10.48  10.30  10.38   4470300  11.25  06-Sep-2018   
673  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.25  06-Sep-2018   
674  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.25  06-Sep-2018   
675  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.25  06-Sep-2018   
676  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.25  06-Sep-2018   
677  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.25  06-Sep-2018   
678  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.25  06-Sep-2018   
679  10.43  07-Sep-2018  10.53  10.36  10.39   5186800  11.43  05-Sep-2018   
680    NaN          NaN    NaN    NaN    NaN       NaN  11.43  05-Sep-2018   
681    NaN          NaN    NaN    NaN    NaN       NaN  11.43  05-Sep-2018   
682    NaN          NaN    NaN    NaN    NaN       NaN  11.43  05-Sep-2018   
683    NaN          NaN    NaN    NaN    NaN       NaN  11.43  05-Sep-2018   
684    NaN          NaN    NaN    NaN    NaN       NaN  11.43  05-Sep-2018   
685    NaN          NaN    NaN    NaN    NaN       NaN  11.43  05-Sep-2018   
686    NaN          NaN    NaN    NaN    NaN       NaN  11.38  04-Sep-2018   
687    NaN          NaN    NaN    NaN    NaN       NaN  11.38  04-Sep-2018   
688    NaN          NaN    N